In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
cd C:\Users\USER\Quant

C:\Users\USER\Quant


 $$ 
 \begin{array}{c}
 y = ln(\frac{K}{S}) \\
 w = {\sigma_{implied}(y,T)}^2  \ T \\ \\
 \frac{\delta w}{\delta y}  = \frac{w[1:]-w[:-1]}{y[1:]-y[:-1]}\ , \
 \frac{\delta^2 w}  {\delta {y}^2 } = \frac{2}{y[2:] - y[:-2]} (\frac{w[2:]-w[1:-1]}{y[2:]-y[1:]} - \frac{w[1:-1] - w[:-2]}{y[1:-1] - y[:-2]})
 \\
 \\ \\ D = 1-\frac{y}{w}\frac{dw}{dy} + \frac{1}{2}\frac{\delta^2 w}{{\delta y}^2} + \frac{1}{4}( \frac{y^2}{w^2}-\frac{1}{w}-\frac{1}{4} ){(\frac{\delta w}{\delta y})}^2 \\ \\ 
 {\sigma_{local}}^2(T,y) = \frac{dw}{dT} \times \frac{1}{D}
 \\ \\ 
 \end{array}
 $$

In [172]:
def calibrate_local_vol(path_option_data, sheet_imvol, sheet_call_price, node = 5) :
    imvol = pd.read_excel(path_option_data, sheet_name = sheet_imvol, index_col = 0)
    call_price = pd.read_excel(path_option_data, sheet_name = sheet_call_price, index_col = 0)
    T = list(pd.Series(call_price.columns).apply(lambda x : x.split('M')[0]).astype(int)/12)
    call_price.columns = T
    imvol.columns = T
    K = np.array(imvol.index).reshape(-1,1)
    dK = np.array(pd.Series(K.reshape(-1)).diff()).reshape(-1,1)
    dT = np.array(pd.Series(T).diff()).reshape(1,-1)
    w = imvol **2 * np.array(T).reshape(1,-1)
    w_T = w.diff(axis = 1)/dT
    y = np.log(K/100)
    dy = np.array(pd.DataFrame(y).diff())
    dw = w.diff()
    #dw__dw_dy__dy = (dw/dy).diff()/dy
    dw__dw_dy__dy = np.zeros(dw.shape)
    fdiff = (np.array(w)[2:] - np.array(w)[1:-1])
    bdiff = (np.array(w)[1:-1] - np.array(w)[:-2])
    fdk = (y[2:]-y[1:-1])
    bdk = (y[1:-1] - y[:-2])
    two_diff = y[2:] - y[:-2]
    ans =  2/two_diff * (fdiff/fdk - bdiff/bdk)
    dw__dw_dy__dy[1:-1] = ans
    denominator = 1- y/w * dw/dy + 0.5 * dw__dw_dy__dy + 1/4 * (y**2/(w**2) - 1/w - 1/4) *(dw/dy)**2
    
    local_vol = np.sqrt((w_T/denominator).applymap(lambda x : np.nan if x < 0 else x).interpolate(method = 'linear'))
    #######################
    ## interpolate imvol ##
    #######################
    data = local_vol[local_vol.columns[1:]].dropna()
    my_data = data.unstack()
    x = np.array(pd.MultiIndex.droplevel(my_data.index,0))
    y = my_data.values **2
    model = sm.OLS(y,sm.add_constant(x)).fit()
    slope = model.params[-1]
    local_variance = local_vol[local_vol.columns[1:]].dropna()**2
    for i in range(10) :    
        data = pd.DataFrame(pd.Series(local_variance.iloc[0]  + (- node * slope)).rename(local_variance.index[0]-node)).T
        if local_variance.index[0] - node == 45 :
            break
        local_variance = pd.concat([data,local_variance],axis = 0)
    for i in range(3) :
        data2 = pd.DataFrame(pd.Series(local_variance.iloc[-1] + (node * slope)).rename(local_variance.index[-1]+node)).T
        if (data2<0).sum().sum() > 0 :
            break    
        local_variance = pd.concat([local_variance,data2],axis = 0)
    lv = np.sqrt(local_variance)
    
    return lv

In [173]:
path_option_data = 'option.xlsx'
sheet_imvol, sheet_call_price = 'SX5E_vol','SX5E_Call_Price'
sheet_imvol2, sheet_call_price2 = 'HSI_vol','HSI_Call_Price'

In [174]:
node = 5

In [184]:
HSI_lv = calibrate_local_vol(path_option_data, sheet_imvol2, sheet_call_price2, node = 5)
HSI_lv

,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.50,3.00
50,0.450217,0.462665,0.403529,0.362567,0.364516,0.373110,0.345986,0.340015,0.364401
55,0.437939,0.450727,0.389784,0.347203,0.349239,0.358199,0.329851,0.323582,0.349119
60,0.425307,0.438463,0.375535,0.331127,0.333261,0.342639,0.312886,0.306269,0.333136
65,0.412288,0.425847,0.360724,0.314230,0.316478,0.326339,0.294945,0.287917,0.316346
70,0.398844,0.412845,0.345279,0.296372,0.298754,0.309181,0.275841,0.268313,0.298614
80,0.349151,0.296452,0.240164,0.237350,0.226113,0.250578,0.229913,0.231302,0.246900
85,0.241808,0.342838,0.204604,1.097642,0.193200,0.224210,0.214127,0.217040,0.225980
90,0.210532,0.188821,0.161389,0.165646,0.212973,0.211179,0.197819,0.207700,0.211611
95,0.178453,0.183200,0.187185,0.194025,0.203021,0.201566,0.192429,0.200927,0.200219
100,0.160049,0.171610,0.182385,0.186265,0.196790,0.197452,0.200022,0.215395,0.200418


In [ ]:
HSI_lv.to_excel('HSI_local_vol.xlsx')